In [1]:
!pip install selenium
!pip install bs4
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 32.5 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import os


In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Запуск в headless-режиме
options.add_argument("--no-sandbox")  # Отключение sandbox
options.add_argument("--disable-dev-shm-usage")  # Решение для ограниченной памяти
options.add_argument("--disable-gpu")  # Отключение GPU
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(options=options)

In [4]:
#scrap the first turtle
try:
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.scrapethissite.com/pages/frames/")
    time.sleep(2)

    #go  to iframe
    frame = driver.find_element(By.ID, "iframe")
    driver.switch_to.frame(frame)
    print("Switched to iframe")

    # swich to the page of species
    learn_more_link = driver.find_element(By.LINK_TEXT, "Learn more →")
    learn_more_link.click()
    time.sleep(2)
    print("Navigated to species page")

    # parse the info
    soup = BeautifulSoup(driver.page_source, "html.parser")
    family_name = soup.select_one("h3.family-name").text.strip()
    description = soup.select_one("p.lead").text.strip()
    image_element = soup.select_one("img.turtle-image.center-block")
    image_url = image_element.get("src") if image_element else None

    # download the pic
    if image_url:
        image_filename = f"{family_name}.jpg"
        image_path = os.path.join("images", image_filename)
        os.makedirs("images", exist_ok=True)
        response = requests.get(image_url, stream=True)
        with open(image_path, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Image saved: {image_path}")

    # saving the data
    data = {
        "Family Name": [family_name],
        "Description": [description],
        "Image URL": [image_url]
    }
    df = pd.DataFrame(data)
    df.to_csv("species_data.csv", index=False)
    print("Data saved to species_data.csv")


    back_link = driver.find_element(By.LINK_TEXT, "← Back to all Turtles")
    back_link.click()
    time.sleep(2)
    print("Returned to iframe page")


    print(df)

except Exception as e:
    print(f"Error: {e}")

finally:
    driver.quit()

Switched to iframe
Navigated to species page
Image saved: images/Carettochelyidae.jpg
Data saved to species_data.csv
Error: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.204)
Stacktrace:
#0 0x571194cf88fa <unknown>
#1 0x571194809d20 <unknown>
#2 0x5711947f0a75 <unknown>
#3 0x5711947f0953 <unknown>
#4 0x57119480c042 <unknown>
#5 0x57119489af46 <unknown>
#6 0x57119487c893 <unknown>
#7 0x57119484b30d <unknown>
#8 0x57119484c32e <unknown>
#9 0x571194cc500b <unknown>
#10 0x571194cc8f97 <unknown>
#11 0x571194cb171c <unknown>
#12 0x571194cc9b17 <unknown>
#13 0x571194c966cf <unknown>
#14 0x571194ce76b8 <unknown>
#15 0x571194ce7880 <unknown>
#16 0x571194cf7776 <unknown>
#17 0x7a3b780deac3 <unknown>



In [5]:
#  make the function based on previous code to scrab species
def scrape_species(driver):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    family_name = soup.select_one("h3.family-name").text.strip()
    description = soup.select_one("p.lead").text.strip()
    image_element = soup.select_one("img.turtle-image.center-block")
    image_url = image_element.get("src") if image_element else None


    if image_url:
        image_filename = f"{family_name}.jpg"
        image_path = os.path.join("images", image_filename)
        os.makedirs("images", exist_ok=True)
        response = requests.get(image_url, stream=True)
        with open(image_path, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Image saved: {image_path}")

    return {
        "Family Name": family_name,
        "Description": description,
        "Image URL": image_url
    }

In [6]:
# this the main function to scrape all the spiecies

def scrape_all_species():
    driver = webdriver.Chrome(options=options)
    extracted_data = []

    try:
        driver.get("https://www.scrapethissite.com/pages/frames/")
        time.sleep(2)

        def switch_to_frame():        #goin into frame
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "iframe")))
                frame = driver.find_element(By.ID, "iframe")
                driver.switch_to.frame(frame)
                print("Switched to iframe")
            except Exception as e:
                print(f"Error switching to iframe: {e}")
                raise


        switch_to_frame()

        #scrab all links witj turtle species
        soup = BeautifulSoup(driver.page_source, "html.parser")
        learn_more_links = soup.select("a.btn.btn-default.btn-xs")

        for link in learn_more_links:
            href = link.get("href")
            full_url = f"https://www.scrapethissite.com{href}"
            print(f"Navigating to: {full_url}")

            try:
                # goung to the page of species
                driver.get(full_url)
                time.sleep(2)

                # collecting the data
                species_data = scrape_species(driver)
                if species_data:
                    extracted_data.append(species_data)

                # pushing the button back
                back_link = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "← Back to all Turtles"))
                )
                back_link.click()
                time.sleep(2)

                # going into frame again
                switch_to_frame()

            except Exception as e:
                print(f"Error processing species page: {e}")
                continue

    except Exception as e:
        print(f"Error in scrape_all_species: {e}")

    finally:
        driver.quit()

    # Save all data
    df = pd.DataFrame(extracted_data)
    df.to_csv("species_data.csv", index=False)
    print("Data saved to species_data.csv")

In [7]:
# Run the progrm
if __name__ == "__main__":
    scrape_all_species()

Switched to iframe
Navigating to: https://www.scrapethissite.com/pages/frames/?frame=i&family=Carettochelyidae
Image saved: images/Carettochelyidae.jpg
Error switching to iframe: Message: 
Stacktrace:
#0 0x57b2b5a758fa <unknown>
#1 0x57b2b5586d20 <unknown>
#2 0x57b2b55d5a66 <unknown>
#3 0x57b2b55d5d01 <unknown>
#4 0x57b2b561b184 <unknown>
#5 0x57b2b55f9b1d <unknown>
#6 0x57b2b5618560 <unknown>
#7 0x57b2b55f9893 <unknown>
#8 0x57b2b55c830d <unknown>
#9 0x57b2b55c932e <unknown>
#10 0x57b2b5a4200b <unknown>
#11 0x57b2b5a45f97 <unknown>
#12 0x57b2b5a2e71c <unknown>
#13 0x57b2b5a46b17 <unknown>
#14 0x57b2b5a136cf <unknown>
#15 0x57b2b5a646b8 <unknown>
#16 0x57b2b5a64880 <unknown>
#17 0x57b2b5a74776 <unknown>
#18 0x7c0652310ac3 <unknown>

Error processing species page: Message: 
Stacktrace:
#0 0x57b2b5a758fa <unknown>
#1 0x57b2b5586d20 <unknown>
#2 0x57b2b55d5a66 <unknown>
#3 0x57b2b55d5d01 <unknown>
#4 0x57b2b561b184 <unknown>
#5 0x57b2b55f9b1d <unknown>
#6 0x57b2b5618560 <unknown>
#7 0x57b

In [9]:
df = pd.read_csv("species_data.csv")
df

,Family Name,Description,Image URL
0,Carettochelyidae,The Carettochelyidae family of turtles — more ...,https://upload.wikimedia.org/wikipedia/commons...
1,Cheloniidae,The Cheloniidae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...
2,Chelydridae,The Chelydridae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...
3,Dermatemydidae,The Dermatemydidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...
4,Dermochelyidae,The Dermochelyidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...
5,Emydidae,The Emydidae family of turtles — more commonly...,https://upload.wikimedia.org/wikipedia/commons...
6,Geoemydidae,The Geoemydidae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...
7,Kinosternidae,The Kinosternidae family of turtles — more com...,https://upload.wikimedia.org/wikipedia/commons...
8,Platysternidae,The Platysternidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...
9,Testudinidae,The Testudinidae family of turtles — more comm...,https://upload.wikimedia.org/wikipedia/commons...
